In [28]:
import json
import numpy as np

class World:
    def __init__(self, grid_size=10, max_world_size=1000):
        self.grid_size = grid_size
        self.max_world_size = max_world_size
        self.grid = {}

    def get_area(self, x, y, z):
        area_x = x // self.grid_size
        area_y = y // self.grid_size
        area_z = z // self.grid_size
        return (area_x, area_y, area_z)

    def set_area_parameters(self, x_1, x_2, y_1, y_2, parameters):
        for x in range(x_1, x_2 + 1, self.grid_size):
            for y in range(y_1, y_2 + 1, self.grid_size):
                for z in range(0, self.max_world_size * self.grid_size, self.grid_size):
                    area = self.get_area(x, y, z)
                    if area not in self.grid:
                        self.grid[area] = {}
                    self.grid[area].update(parameters)

    def get_area_parameters(self, x, y, z):
        area = self.get_area(x, y, z)
        return self.grid.get(area, {})

    def get_area_center_point(self, x, y, z):
        area_x, area_y, area_z = self.get_area(x, y, z)
        center_x = (area_x * self.grid_size) + (self.grid_size / 2)
        center_y = (area_y * self.grid_size) + (self.grid_size / 2)
        center_z = (area_z * self.grid_size) + (self.grid_size / 2)
        return (center_x, center_y, center_z)

    def save_world(self, filename):
        with open(filename, 'w') as file:
            json.dump(self.grid, file)

    def load_world(self, filename):
        with open(filename, 'r') as file:
            self.grid = json.load(file)

class Drone:
    def __init__(self, model_name, rpm_values, x, y, z):
        self.model_name = model_name
        self.rpm_values = rpm_values  # List of 4 RPM values for the rotors
        self.position = np.array([x, y, z], dtype=float)
        self.velocity = np.array([0.0, 0.0, 0.0], dtype=float)
        self.acceleration = np.array([0.0, 0.0, 0.0], dtype=float)
        self.update_acceleration()

    def update_rpm_for_target(self, target_point, kp_xy=0.1, kd_xy=0.05, kp_z=0.1, kd_z=0.05):
        """
        Adjust rotor speeds based on the positional error, with a PD controller on each axis.
        For a quadcopter arranged as:
          - Rotor0: front-left
          - Rotor1: front-right
          - Rotor2: rear-left
          - Rotor3: rear-right

        The horizontal (x, y) control:
          - acceleration_x = 0.02 * a, so a = (kp_xy*error_x - kd_xy*velocity_x)/0.02.
          - acceleration_y = 0.02 * b, so b = (kp_xy*error_y - kd_xy*velocity_y)/0.02.

        Vertical control:
          We want zero vertical acceleration when the average rotor speed is 2500.
          Our vertical acceleration is computed as:
             acceleration_z = ((average_rpm - 2500) * 0.01).
          So we set:
             desired_a_z = kp_z * error_z - kd_z * velocity_z,
             and require (base - 2500)*0.01 = desired_a_z, i.e.
             base = 2500 + desired_a_z/0.01.
        """
        error = target_point - self.position

        # Horizontal control (x and y)
        desired_a_x = kp_xy * error[0] - kd_xy * self.velocity[0]
        desired_a_y = kp_xy * error[1] - kd_xy * self.velocity[1]
        a = desired_a_x / 0.02  # pitch adjustment
        b = desired_a_y / 0.02  # roll adjustment

        # Vertical control (z)
        desired_a_z = kp_z * error[2] - kd_z * self.velocity[2]
        # When error is 0 and velocity is 0, we want base to be 2500.
        base = 2500 + desired_a_z / 0.01

        # Update rotor speeds.
        self.rpm_values[0] = base + a + b   # front-left
        self.rpm_values[1] = base + a - b   # front-right
        self.rpm_values[2] = base - a + b   # rear-left
        self.rpm_values[3] = base - a - b   # rear-right

    def update_acceleration(self):
        # X-axis: difference between front and rear average rotor speeds.
        front_rpm = (self.rpm_values[0] + self.rpm_values[1]) / 2
        rear_rpm = (self.rpm_values[2] + self.rpm_values[3]) / 2
        self.acceleration[0] = (front_rpm - rear_rpm) * 0.01

        # Y-axis: difference between left and right average rotor speeds.
        left_rpm = (self.rpm_values[0] + self.rpm_values[2]) / 2
        right_rpm = (self.rpm_values[1] + self.rpm_values[3]) / 2
        self.acceleration[1] = (left_rpm - right_rpm) * 0.01

        # Z-axis: we subtract 2500 so that 2500 is our hover equilibrium.
        average_rpm = sum(self.rpm_values) / 4
        self.acceleration[2] = (average_rpm - 2500) * 0.01

    def update_velocity(self, time_step):
        self.velocity += self.acceleration * time_step

    def update_position(self, time_step):
        self.position += self.velocity * time_step

class Simulation:
    def __init__(self, drone: Drone, world: World):
        self.drone = drone
        self.world = world

    def generate_intermediate_points(self, point_a, point_b, num_points=10):
        points = [point_a]
        for i in range(1, num_points + 1):
            intermediate_point = point_a + (point_b - point_a) * (i / (num_points + 1))
            points.append(intermediate_point)
        points.append(point_b)
        return points

    def simulate_trajectory(self, point_a, point_b, time_step=0.1, threshold=1.0):
        point_a = np.array(point_a, dtype=float)
        point_b = np.array(point_b, dtype=float)
        intermediate_points = self.generate_intermediate_points(point_a, point_b)
        trajectory = [self.drone.position.copy()]
        time_passed = 0

        for target_point in intermediate_points:
            while np.linalg.norm(self.drone.position - target_point) > threshold:
                # Update rotor speeds using our PD controller.
                self.drone.update_rpm_for_target(target_point)
                # Recompute acceleration from the new rotor speeds.
                self.drone.update_acceleration()
                # Update velocity and position.
                self.drone.update_velocity(time_step)
                self.drone.update_position(time_step)
                trajectory.append(self.drone.position.copy())
                time_passed += time_step
                print(f"Time: {time_passed:6.2f}s | Position: {self.drone.position} | RPMS: {self.drone.rpm_values}")
        print()  # Newline after simulation loop
        return trajectory

# Example usage:
if __name__ == "__main__":
    world = World(grid_size=10, max_world_size=1000)
    world.set_area_parameters(x_1=0, x_2=20, y_1=0, y_2=20, parameters={'temperature': 20, 'humidity': 50})

    # Start with an initial RPM of 2500 for all four rotors.
    drone = Drone(model_name="Quadcopter", rpm_values=[2500, 2500, 2500, 2500], x=0, y=0, z=0)
    simulation = Simulation(drone, world)

    # Target point (100, 100, 100)
    trajectory = simulation.simulate_trajectory(point_a=[0, 0, 0], point_b=[100, 100, 250])

    # Print the final trajectory (or you could plot it)
    print("\nFinal Trajectory:")
    for point in trajectory:
        print(point)


Time:   0.10s | Position: [0.00909091 0.00909091 0.02272727] | RPMS: [2818.1818181818185, 2727.2727272727275, 2727.2727272727275, 2636.3636363636365]
Time:   0.20s | Position: [0.02721818 0.02721818 0.06804545] | RPMS: [2816.272727272727, 2725.909090909091, 2725.909090909091, 2635.545454545455]
Time:   0.30s | Position: [0.05431851 0.05431851 0.13579627] | RPMS: [2814.0569090909094, 2724.3263636363636, 2724.3263636363636, 2634.595818181818]
Time:   0.40s | Position: [0.09031993 0.09031993 0.22579981] | RPMS: [2811.538113090909, 2722.5272236363635, 2722.5272236363635, 2633.516334181818]
Time:   0.50s | Position: [0.13514192 0.13514192 0.33785481] | RPMS: [2808.720372957818, 2720.5145521127274, 2720.5145521127274, 2632.3087312676366]
Time:   0.60s | Position: [0.18869558 0.18869558 0.47173895] | RPMS: [2805.608001152435, 2718.2914293945964, 2718.2914293945964, 2630.974857636758]
Time:   0.70s | Position: [0.25088368 0.25088368 0.6272092 ] | RPMS: [2802.2055832049264, 2715.8611308606614, 